# Part 4: Real-Time Prediction

**Objective**: Use our saved batch model to score a live simulated stream. This provides the "wow" moment with zero setup.


In [ ]:
# Setup: Import required libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *
import mlflow

## Module 4.1: The "Live Order" Simulator

**Goal**: Use the rate source (a built-in simulator) to manufacture a new stream of orders.


In [ ]:
# Load the trained model from Part 3
runs = mlflow.search_runs()
latest_run = runs.iloc[0]
run_id = latest_run['run_id']

model_uri = f"runs:/{run_id}/my_tpch_order_value_model"
loaded_model = mlflow.spark.load_model(model_uri)

print(f"✓ Model loaded!")


### Create a Simulated Stream

The `rate` source generates a stream of timestamps at a specified rate - perfect for simulating live data!


In [ ]:
# Create rate stream (generates timestamps)
rate_stream = spark.readStream \
    .format("rate") \
    .option("rowsPerSecond", 1) \
    .load()

rate_stream.printSchema()


In [ ]:
# Transform rate stream into order data (simple simulation)
orders_stream = rate_stream.select(
    col("timestamp").alias("order_time"),
    # Simulate the 3 features our model needs
    (rand() * 12 + 1).cast("int").alias("month"),
    (rand() * 100000 - 1000).cast("double").alias("c_acctbal"),
    # Market segment
    when(rand() > 0.8, "AUTOMOBILE")
    .when(rand() > 0.6, "BUILDING")
    .when(rand() > 0.4, "MACHINERY")
    .when(rand() > 0.2, "HOUSEHOLD")
    .otherwise("FURNITURE").alias("c_mktsegment")
)

orders_stream.printSchema()


## Module 4.2: Apply Model & Display Live

**Goal**: See live predictions in the notebook.


### Prepare Features for the Model

We need to apply the same feature engineering pipeline that was used during training.


In [ ]:
# The loaded model is a full pipeline that includes feature engineering!
# It will automatically apply StringIndexer and VectorAssembler
# We just need to provide the raw features matching the pipeline input


### Apply Model to Stream

The model can be applied directly to streaming DataFrames!


In [ ]:
# Apply model to stream (pipeline handles feature engineering automatically!)
predictions_stream = loaded_model.transform(orders_stream)


### Display Live Predictions

Use `display()` (Databricks) or write to console/sink for live predictions in Jupyter notebooks!


In [ ]:
# Display live predictions
# In Databricks: use display() for live updates
# In local: use console sink
query = predictions_stream.select(
    "order_time",
    "c_mktsegment",
    "c_acctbal",
    "prediction"
).writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# In Databricks, use: display(predictions_stream.select("order_time", "c_mktsegment", "c_acctbal", "prediction"))

print("Streaming started! Check console for predictions.")
print("To stop: query.stop()")


### Live Predictions!

**What you're seeing**:
- A stream generating predictions every second
- Each row is a new order being scored in real-time
- The `prediction` column shows predicted order value

**In production**: Real data from Kafka/Kinesis, running 24/7


### 🎯 Key Takeaways

1. **Streaming**: Use `rate` for simulation, Kafka/Kinesis for production
2. **Models**: ML models work seamlessly with streaming DataFrames
3. **Real-Time**: Same batch model scores live data
4. **Display**: Use `display()` in Databricks for live updates


### 💡 Production Considerations

**For real production streaming**:
- Use Kafka, Kinesis, or Event Hub as source
- Write predictions to Delta Lake or database
- Enable checkpointing for fault tolerance
- Monitor with Spark UI and alerting

**Example production code**:
```python
query = predictions_stream.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/checkpoint/path") \
    .outputMode("append") \
    .start("dbfs:/predictions/")
```


---

## 🎉 Congratulations! Workshop Complete!

### What You've Built Today

In just 2 hours, you've created a **production-grade, end-to-end data pipeline**:

1. ✅ **PySpark Fundamentals**
   - Mastered lazy evaluation (transformations vs actions)
   - Learned DataFrame API (select, filter, withColumn, groupBy)
   - Used Spark UI for debugging

2. ✅ **Data Engineering Skills**
   - Performed complex joins and aggregations
   - Cleaned and transformed large datasets
   - Saved data to Delta Lake (ACID transactions + time travel)

3. ✅ **Machine Learning Pipeline**
   - Built feature engineering pipeline (StringIndexer, VectorAssembler)
   - Trained RandomForestRegressor for predictive analytics
   - Tracked experiments with MLflow (automatic versioning)

4. ✅ **Real-Time Streaming**
   - Created simulated streaming data source
   - Applied batch ML model to live stream
   - Implemented real-time prediction system

### 💼 Resume-Worthy Achievement

You can now confidently add to your resume:

> **"Built an end-to-end ML pipeline with PySpark and MLlib on Databricks, including feature engineering, model training with MLflow experiment tracking, and real-time prediction using Structured Streaming. Processed large-scale datasets with distributed computing and deployed production-ready data pipelines using Delta Lake."**

This is **real, industry-relevant experience** that employers value!

---

## 🚀 Next Steps: Your Learning Journey

### Week 1: Solidify Your Skills
- [ ] **Re-run the workshop** on your own (muscle memory!)
- [ ] **Experiment**: Change model parameters, add more features
- [ ] **Try different datasets**: Explore `/databricks-datasets/`
- [ ] **Modify the pipeline**: Use different ML algorithm (GBT, Logistic Regression)

### Month 1: Expand Your Knowledge

#### Free Learning Resources
1. **Databricks Academy** (100% free courses):
   - "Apache Spark Programming with Databricks" (4 hours)
   - "Data Engineering with Databricks" (6 hours)
   - "Machine Learning on Databricks" (5 hours)
   - Link: https://academy.databricks.com/

2. **Practice Projects**:
   - Customer segmentation with K-Means clustering
   - Churn prediction with Gradient Boosted Trees
   - Time series forecasting (IoT sensor data)
   - Recommendation system with ALS (MovieLens dataset)

3. **Read Documentation**:
   - PySpark API: https://spark.apache.org/docs/latest/api/python/
   - MLlib Guide: https://spark.apache.org/docs/latest/ml-guide.html
   - Delta Lake: https://docs.delta.io/

#### Suggested Mini-Projects
```python
# Project 1: Customer Segmentation (2-3 hours)
# Dataset: /databricks-datasets/retail-org/customers/
# Goal: Use K-Means to segment customers by behavior
# Skills: Feature engineering, clustering, visualization

# Project 2: Flight Delay Prediction (3-4 hours)
# Dataset: /databricks-datasets/airlines/
# Goal: Predict flight delays using classification
# Skills: Multi-class classification, cross-validation, hyperparameter tuning

# Project 3: IoT Anomaly Detection (4-5 hours)
# Dataset: /databricks-datasets/iot/
# Goal: Detect anomalous sensor readings in real-time
# Skills: Streaming, anomaly detection, windowing
```

### Month 2-3: Build Portfolio & Get Certified

#### Build Portfolio Projects
1. **Create a public GitHub repo**:
   - Well-documented README with business context
   - Jupyter notebooks or scripts
   - Results visualization (charts, dashboards)
   - Clear instructions to reproduce

2. **Project Ideas**:
   - NYC Taxi: Trip duration prediction + surge pricing analysis
   - E-commerce: Market basket analysis + product recommendation
   - Social media: Sentiment analysis + trend detection
   - Healthcare: Patient readmission risk prediction

3. **Share Your Work**:
   - LinkedIn posts with visualizations
   - Medium/Dev.to blog posts
   - Present at local meetups
   - Contribute to open-source Spark projects

#### Pursue Certification
- **Databricks Certified Data Engineer Associate**
  - Cost: $200
  - Study guide: https://databricks.com/learn/certification
  - Practice exams available
  
- **Databricks Certified ML Associate**
  - Focuses on MLlib and MLflow
  - Great for ML engineers and data scientists

### Advanced Topics to Explore

#### Performance Optimization
```python
# Broadcast joins for small tables
from pyspark.sql.functions import broadcast
large_df.join(broadcast(small_df), "id")

# Repartitioning for better parallelism
df.repartition(100, "customer_id")

# Z-ordering for Delta Lake
spark.sql("OPTIMIZE delta.`/path/` ZORDER BY (customer_id, date)")

# Adaptive Query Execution (AQE)
spark.conf.set("spark.sql.adaptive.enabled", "true")
```

#### Advanced Streaming
```python
# Stateful streaming with watermarking
streaming_df.withWatermark("timestamp", "10 minutes") \
    .groupBy(window("timestamp", "5 minutes"), "user_id") \
    .count()

# Stream-to-stream joins
stream1.join(stream2, "user_id", "inner")

# Kafka integration (production)
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "host:port") \
    .option("subscribe", "topic") \
    .load()
```

#### Delta Lake Advanced Features
```python
# Time travel (query old versions)
df = spark.read.format("delta") \
    .option("versionAsOf", 5) \
    .load("/path/to/delta")

# Merge (upsert) operations
from delta.tables import DeltaTable
delta_table = DeltaTable.forPath(spark, "/path/")
delta_table.merge(updates, "target.id = source.id") \
    .whenMatchedUpdate(set = {"value": "source.value"}) \
    .whenNotMatchedInsert(values = {"id": "source.id", "value": "source.value"}) \
    .execute()

# Schema evolution
df.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .save("/path/")
```

---

## 🎓 Career Pathways

### Roles You're Now Prepared For

1. **Data Engineer** (Entry-Level)
   - Build and maintain data pipelines
   - ETL/ELT development
   - Data quality monitoring
   - Average salary: $85k-$110k

2. **Analytics Engineer**
   - Transform raw data for BI/analytics
   - Write SQL and PySpark for transformations
   - Collaborate with analysts and data scientists
   - Average salary: $80k-$105k

3. **ML Engineer** (Junior)
   - Deploy ML models to production
   - Build feature pipelines
   - Monitor model performance
   - Average salary: $90k-$120k

4. **Data Scientist** (with engineering skills)
   - Analyze data at scale with PySpark
   - Build and deploy ML models
   - Communicate insights to stakeholders
   - Average salary: $95k-$130k

### Skills to Add Next

**For Data Engineers:**
- Airflow/Databricks Workflows (orchestration)
- dbt (transformation framework)
- CI/CD for data pipelines
- Data quality frameworks (Great Expectations)
- Cloud platforms (AWS/Azure/GCP)

**For ML Engineers:**
- Model serving (MLflow, FastAPI, Kubernetes)
- A/B testing frameworks
- Model monitoring (drift detection)
- Feature stores (Databricks Feature Store, Feast)
- Advanced MLOps practices

**For Data Scientists:**
- Deep learning (TensorFlow, PyTorch)
- Advanced algorithms (XGBoost, LightGBM)
- Statistical analysis (hypothesis testing, causal inference)
- Experimentation platforms
- Data visualization (Tableau, Power BI)

---

## 🌟 Community & Support

### Join the Community

**Databricks Community:**
- Forums: https://community.databricks.com/
- LinkedIn: Follow #Databricks, #ApacheSpark
- Twitter: @databricks, @apachespark
- Slack: Various Spark communities

**Open Source:**
- Apache Spark: https://spark.apache.org/community.html
- Contribute to Spark projects on GitHub
- Answer questions on Stack Overflow (tag: `pyspark`, `databricks`)

**Local Meetups:**
- Search Meetup.com for "Spark" or "Big Data"
- Attend virtual conferences (Spark + AI Summit)
- Join data engineering Discord servers

### Stay Updated

**Blogs to Follow:**
- Databricks Blog: https://databricks.com/blog
- Apache Spark Blog: https://spark.apache.org/news/
- Towards Data Science (Medium)
- KDnuggets

**Podcasts:**
- Data Engineering Podcast
- The Data Stack Show
- Software Engineering Daily (data episodes)

**YouTube Channels:**
- Databricks
- Advancing Analytics
- Data Council

---

## 📊 Workshop Feedback

We'd love to hear from you! Your feedback helps us improve.

**What worked well?**
- What concepts clicked?
- What exercises were most valuable?
- What should we keep?

**What could be better?**
- What was confusing?
- What topics need more depth?
- What would you like to see added?

**Share your feedback:**
- GitHub issues: [Link to repo]
- Email: [instructor email]
- Survey: [Link to survey]

**Share your success:**
- Built a project? Share it on LinkedIn! Tag us
- Got certified? Celebrate with the community
- Found a job? Let us know - we'd love to hear!

---

## 🏆 Final Thoughts

### You've Accomplished Something Real

Many tutorials teach theory. You built a **real system**:
- Processed large-scale data efficiently
- Trained a production-quality ML model
- Deployed real-time predictions
- Used industry-standard tools (Databricks, MLflow, Delta)

This is **the same stack** used by:
- Netflix (recommendation systems)
- Uber (surge pricing, ETAs)
- Airbnb (pricing optimization)
- Microsoft (Azure analytics)
- Thousands of other companies

### The Journey Continues

Becoming proficient with Spark is a journey. You've taken the first big step today:
- ✅ You understand the fundamentals
- ✅ You've written real code
- ✅ You've built an end-to-end pipeline
- ✅ You have a foundation to build on

**Next step: Build something.** The best learning happens when you're solving real problems.

### Remember

> **"The expert in anything was once a beginner."**

You started as a beginner this morning. You're already further than 95% of people who say "I should learn Spark someday."

**Keep going. Keep building. Keep learning.**

---

## 🎓 Additional Resources Checklist

Save these links for future reference:

### Documentation
- [ ] PySpark API: https://spark.apache.org/docs/latest/api/python/
- [ ] Databricks Docs: https://docs.databricks.com/
- [ ] Delta Lake: https://docs.delta.io/
- [ ] MLflow: https://mlflow.org/docs/latest/index.html

### Learning Platforms
- [ ] Databricks Academy: https://academy.databricks.com/
- [ ] Coursera: "Big Data Analysis with Scala and Spark"
- [ ] Udemy: Search "PySpark" (check ratings!)
- [ ] YouTube: Databricks channel

### Practice
- [ ] Kaggle competitions (use PySpark for large datasets)
- [ ] Databricks sample notebooks: https://docs.databricks.com/notebooks/gallery.html
- [ ] GitHub: Search "pyspark projects" for inspiration

### Reference Materials
- [ ] This workshop repo (star it!)
- [ ] PySpark Cheat Sheet: [PYSPARK_CHEATSHEET.md](PYSPARK_CHEATSHEET.md)
- [ ] Workshop Guide: [WORKSHOP_GUIDE.md](WORKSHOP_GUIDE.md)
- [ ] Databricks Setup: [DATABRICKS_FREE_EDITION_GUIDE.md](DATABRICKS_FREE_EDITION_GUIDE.md)

---

## 🚀 Your Action Plan (Next 7 Days)

### Day 1 (Today): Solidify
- [ ] Re-run all 4 notebooks on your own
- [ ] Experiment: Change one parameter in the ML model
- [ ] Save notebooks to your local machine (backup)

### Day 2-3: Explore
- [ ] Load a different Databricks dataset
- [ ] Perform basic analysis (groupBy, aggregations)
- [ ] Create simple visualizations

### Day 4-5: Build
- [ ] Start a mini-project (2-3 hours)
- [ ] Use a different ML algorithm (try GBTRegressor)
- [ ] Save results to your GitHub

### Day 6-7: Share
- [ ] Write a LinkedIn post about what you learned
- [ ] Share your mini-project
- [ ] Connect with 3 people in the data community

**Set a calendar reminder now!** Don't let momentum fade.

---

## 🎉 Congratulations Again!

You did it! You spent 2 hours investing in yourself and learning valuable, in-demand skills.

**From all of us: Thank you for your time, attention, and curiosity.**

**Now go build something amazing with PySpark! ⚡**

---

**Questions? Reach out anytime:**
- GitHub Issues: [repo link]
- Community Forums: https://community.databricks.com/
- LinkedIn: [instructor profile]

**Happy Sparking! 🚀✨**


In [ ]:
# Stop the streaming query when done
# Uncomment the line below to stop the stream
query.stop()
